In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
import tempfile
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import norm
import tqdm
import pickle
from itertools import combinations
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.optics import euclidean_distance

from PIL import Image, ImageDraw
from multiprocessing import Pool, Manager
import copy
import uuid
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', 500)





In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

In [ ]:
sql_query = '''
    select * from keypoint_annotations where pen_id=7;
'''
df = rds_access_utils.extract_from_database(sql_query)

In [ ]:
def display_edge(image_url):
    image_bucket = 'aquabyte-crops'
    image_key = image_url[image_url.index('environment'):]
    image_f = s3_access_utils.download_from_s3(image_bucket, image_key)

    img = cv2.imread(image_f, 0)
    img2 = cv2.GaussianBlur(img, (15, 15), 0)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(20,20))
    img3 = clahe.apply(img2)
    plt.figure(figsize=(20, 10))
    edges = cv2.Canny(img3,20,80)

    plt.imshow(edges, cmap = 'gray')

    plt.show()

In [ ]:
display_edge(df.left_image_url.iloc[8])

In [ ]:
display_edge(df.left_image_url.iloc[33])